In [ ]:
#@title 
catalogueEndpoint = 'http://qcmms-cat.spacebel.be/eo-catalog'
lpProductIdentifier='ProductIdentifierToBeReplaced'
lpParentIdentifier= 'ParentIdentifierToBeReplaced'
#lpProductIdentifier='IMD_2018_010m'
#lpParentIdentifier= 'EOP:ESA:LP:UC1'
#lpProductIdentifier='TCCM_1518_020mD'
#lpParentIdentifier= 'EOP:ESA:LP:UC2D'
debug = False
#@title Basic functions
import json, requests
import pandas as pd
import math
import matplotlib.pyplot as plt

from IPython.display import display, HTML
import ipywidgets as widgets

lpUrl=catalogueEndpoint + '/'+ 'series'+'/' + lpParentIdentifier + '/datasets/'+lpProductIdentifier
ipUrl=''
lpMd={}
platforms=[]
fgNbr = 0

processingLevelDefs=["1C","L1TP","L2A", "L2"]
maxRecords = 10

pd.set_option('display.html.table_schema', True)

def getMetric(metric,feature):
    result={}
    metrics=feature['properties']['productInformation']['qualityInformation']['qualityIndicators']
    lenMetrics= len(metrics)
    for i in range(lenMetrics):
      if (metrics[i]['isMeasurementOf'] == metric): 
        result=metrics[i]
    if (result == {} and debug):
      print("metric ", metric,"not found in ", feature['properties']['identifier'] )
    return result

def getProcessingLevelIndex(processingLevel):
  index=-1
  for i in range(len(processingLevelDefs)):
    if (processingLevelDefs[i] == processingLevel ): 
      index=i
  return index

def generateFigureLabel(fgNbr,label):
  lfgNbr=fgNbr+1
  display(widgets.HTML(
    value='<i>Figure '+ str(lfgNbr) +': ' + label+'</i>'
    ))
  return lfgNbr

def getFigureLabel(fgNbr,label):
  return 'Figure '+ str(fgNbr) +': ' + label

tableNbr=0
def generateTableLabel(tableNbr,label):
  lNbr=tableNbr+1
  display(widgets.HTML(
    value='<i>Table '+ str(lNbr) +': ' + label+'</i>'
    ))
  return lNbr

def displayImage(url):
  display(widgets.HTML(
  value= '<img width="500" height="500" src="'+url+'">' 
    ))
#@title Retrieving Land Product
def getLandProduct():
  querystring={}
  nbLp=0
  lpMd={}
  #print(lpUrl)
  response = requests.get(lpUrl, params=querystring)
  pyresult = json.loads(response.text)
  try:
    nbLp  = len(pyresult['features'])
  except KeyError:
    pass
  if (nbLp == 0):
    print("Land Product " + lpProductIdentifier + "("+ lpParentIdentifier +") is not found" )
  else:
    lpMd = pyresult['features'][0]
  return lpMd

def getLpAttribute(lpMdProperties,attribute1,attribute2):
  res=''
  try:
    if (attribute2 != ''):
      res=lpMdProperties[attribute1][attribute2]
    else:
      res=lpMdProperties[attribute1]
  except KeyError:
        pass
  return res 
lpMd=getLandProduct()
#@title Retrieving Related Image Products
def initImageProductsUrl():
  url=''
  lp=getLandProduct()
  if lp != {}:
    try:
      url=lp['properties']['links']['via'][0]['href']
    except KeyError:
      print('Missing via links to Image Products the Land Product') 
  return url
ipUrl=initImageProductsUrl()
#@title Retrieving Related Platforms
def initPlatforms(lpMd):
  nb_platforms=len(lpMd['properties']['acquisitionInformation'])
  lplatforms= [''] * nb_platforms    
  for i in range(len(lpMd['properties']['acquisitionInformation'])):
      lplatforms[i]=lpMd['properties']['acquisitionInformation'][i]['platform']['platformShortName']
  return lplatforms
platforms=initPlatforms(lpMd)

def harmonized():
# nbr of platforms is used to decide if we are in multisensor UC.
# if yes there some additional information for the harmonization.
  if (len(platforms) > 1):
    return True;
  else:
    return False;


def countImageProductByMetric(metric,metricValue):
   # metric can take values
   # degradedDataPercentageMetric		
   # degradedAncillaryDataPercentageMetric		
   # formatCorrectnessMetric
   # generalQualityMetric		
   # geometricQualityMetric		
   # radiometricQualityMetric		
   # sensorQualityMetric	
   # feasibilityControlMetric		
   # deliveryControlMetric		
   # ordinaryControlMetric		
   # detailedControlMetric
  querystring={}
  querystring['maximumRecords'] = 1
  if metric != '':
    querystring['specificationTitle'] = metric
    querystring['degree'] = metricValue
  response = requests.get(ipUrl, params=querystring)
  pyresult= json.loads(response.text)
  return pyresult['totalResults']
#
# build status for each steps
#
#
# return True if the metric is associated to LP 
# check if metric contains lp
def lpMetric(metric):
  if metric.lower().rfind('lp') == -1:
    return False
  else:
    return True

def  getLpMetricStatus(metricName):
  metricPrefix='http://qcmms.esa.int/quality-indicators/#'
  metric=getMetric(metricPrefix+metricName,lpMd)
  if metric != '{}':
    executed=True
  else:
    executed=False
  applicable=True
  if metricName == 'lpInterpretationMetric' and harmonized(): 
    applicable=False
  
  nbPass=0
  nbFailed=0
  try:
    if metric['value']:
      nbPass=1
    else:
      nbFailed=1
  except KeyError:
    print("key error value",metric)
    pass
  result={'Applicable': applicable, 'Executed' : executed, 'PASS': nbPass, 'FAILED' : nbFailed }
  return result

def getMetricStatus(metric):
  if not lpMetric(metric):
    nbPass=countImageProductByMetric(metric,'true')
    nbFailed=countImageProductByMetric(metric,'false')
    applicable=True
    if not harmonized() and metric == 'harmonizationControlMetric':
      applicable=False
    
    if nbPass == 0 and nbFailed == 0:
      executed=False
    else:
      executed=True
    return {'Applicable': applicable, 'Executed' : executed, 'PASS': nbPass, 'FAILED' : nbFailed }
  else:
    return getLpMetricStatus(metric)


MetricsStatusVct=[
                  {'Applicable': True, 'Executed' : True, 'PASS': 1, 'FAILED' :0 },
                  getMetricStatus('feasibilityControlMetric'),
                  getMetricStatus('deliveryControlMetric'),
                  getMetricStatus('ordinaryControlMetric'),
                  getMetricStatus('detailedControlMetric'),
                  getMetricStatus('harmonizationControlMetric'),
                  getMetricStatus('ipForLpInformationMetric'),
                  getMetricStatus('lpInterpretationMetric'),
                  getMetricStatus('lpMetadataControlMetric'),
                  getMetricStatus('lpOrdinaryControlMetric'),
                  getMetricStatus('lpThematicValidationMetric')
                 ]
IndexFeasibility=1
IndexDelivery=2
IndexOrdinary=3
IndexDetailed=4
IndexHarmonization=5
IndexIpForLp=6
IndexLpInterpretation=7
IndexLpMetadata=8
IndexLpOrdinary=9
IndexLpThematic=10
#MetricsStatusVct[IndexFeasibility]['Executed']=False
#MetricsStatusVct[IndexDelivery]['Applicable']=False


def alreadyExecuted(index):
  if not MetricsStatusVct[index]['Applicable']:
    print("This step is not applicable.")
    return False
  if not MetricsStatusVct[index]['Executed']:
    print("This step has not yet been executed.")
    return False
  return True
#
# compute the nbr of PASS/FAILED  for overall assessment and to decide if a step has already been executed.
#

IPsMetricsStatusVct=[
                     {'Metric' :'1. FeasibilityControl',  'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexFeasibility]['PASS']},	
                     {'Metric' :'1. FeasibilityControl',  'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexFeasibility]['FAILED'] },
                     {'Metric' :'2. DeliveryControl',     'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexDelivery]['PASS'] },	
                     {'Metric' :'2. DeliveryControl',     'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexDelivery]['FAILED'] },
                     {'Metric' :'3a. OrdinaryControl',    'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexOrdinary]['PASS'] },
                     {'Metric' :'3a. OrdinaryControl',    'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexOrdinary]['FAILED'] },	
                     {'Metric' :'3b. DetailedControl',    'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexDetailed]['PASS'] },	
                     {'Metric' :'3b. DetailedControl',    'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexDetailed]['FAILED'] }	]
if MetricsStatusVct[IndexHarmonization]['Applicable']:
  IPsMetricsStatusVct.append({'Metric' :'4a. HarmonizationControl',    'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexHarmonization]['PASS'] })
  IPsMetricsStatusVct.append({'Metric' :'4a. HarmonizationControl',    'Value' : 'FAILED' ,  'Nbr' : MetricsStatusVct[IndexHarmonization]['FAILED']  })

def generateDefinitions():
  def0= '<tr><td>'+'Circular Error at the 90th percentile' + '</td><td>'+ 'A minimum of 90 percent of the point are included in the stated CE90 circle' + '</td></tr>'
  def1= '<tr><td>'+'Image Product' + '</td><td>'+ 'Satellite image data with metadata used as input to Land Mapping Production.' + '</td></tr>'
  def2= '<tr><td>'+'Land Product'+ '</td><td>' + 'Land Mapping or Monitoring Product as e.g. Land Cover Map.'+'</td></tr>' 
  def3= '<tr><td>'+'Land Product Specification Table'+'</td><td>'+ 'Set of parameters that defines the Land Product and the required Image Products.'+'</td></tr>' 
  def4= '<tr><td>'+'Root Mean Square Error' +'</td><td>'+  'The standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit.' +'</td></tr>' 
  definitions= '<table border=1 width=80%>' + def0 + def1 + def2 + def3 + '</table>'
  display(widgets.HTML(
    value=definitions
    ))

def generateAbbreviations():
  def0= '<tr><td>'+'CE90' + '</td><td>'+ 'Circular Error at the 90th percentile' + '</td></tr>'
  def1= '<tr><td>'+'EO' + '</td><td>'+ 'Earth Observation' + '</td></tr>'
  def2= '<tr><td>'+'HR' + '</td><td>'+ 'High Resolution' + '</td></tr>'
  def3= '<tr><td>'+'HRL' + '</td><td>'+ 'High Resolution Layer' + '</td></tr>'
  def4= '<tr><td>'+'IP' + '</td><td>'+ 'Image Product' + '</td></tr>'
  def5= '<tr><td>'+'INSPIRE' + '</td><td>'+ 'Infrastructure for Spatial Information in Europe' + '</td></tr>'
  def6= '<tr><td>'+'LCCM' + '</td><td>'+ 'Land Cover Continuous Monitoring' + '</td></tr>'
  def7= '<tr><td>'+'LP'+  '</td><td>' + 'Land Product' + '</td></tr>'
  def8= '<tr><td>'+'MMU'+  '</td><td>' + 'Minimum Mapping Unit' + '</td></tr>'
  def9= '<tr><td>'+'RMSE'+  '</td><td>' + 'Root Mean Square Error' + '</td></tr>'
  def10= '<tr><td>'+'VHR'+  '</td><td>' + 'Very High Resolution' + '</td></tr>'
  definitions= '<table border=1 width=80%>' + def0+ def1 + def2 + def3 + def4 + def5 + def6 + def7+ def8+ def9+ def10+'</table>'
  display(widgets.HTML(
    value=definitions
    ))

#@title Generate Land Product Definition
def generate_lp_definition_report():
  table_title =['Parameter','Value/Description']
  
  table_report = []
  lpMd = getLandProduct()
  if(lpMd != {}):
    abstract=lpMd['properties']['abstract']
    display(widgets.HTML(
       value= abstract ))
       
    table_report=[
      ['Product',getLpAttribute(lpMd['properties'],'title','')],
      ['Product type',getLpAttribute(lpMd['properties'],'additionalAttributes','product_type')],   
      ['Geometric resolution',getLpAttribute(lpMd['properties'],'additionalAttributes','geometric_resolution')],
      ['Target CRS',getLpAttribute(lpMd['properties'],'additionalAttributes','crs')], 
      ['Geometric accuracy',getLpAttribute(lpMd['properties'],'additionalAttributes','geometric_accuracy')], 
      ['Thematic accuracy',getLpAttribute(lpMd['properties'],'additionalAttributes','thematic_accuracy')], 
      ['MMU',getLpAttribute(lpMd['properties'],'additionalAttributes','mmu_pixels')],  
      ['Attributes',getLpAttribute(lpMd['properties'],'additionalAttributes','attributes')],
      ['Raster coding',getLpAttribute(lpMd['properties'],'additionalAttributes','raster_coding')],
      ['Coverage area',getLpAttribute(lpMd['properties'],'additionalAttributes','coverage_area')],
      ['Reference year',getLpAttribute(lpMd['properties'],'additionalAttributes','reference_year')],
      ['Temporal window',getLpAttribute(lpMd['properties'],'additionalAttributes','temporal_window')],
      ['Primary image resource',getLpAttribute(lpMd['properties'],'additionalAttributes','primary_image_source')]              
    ]
    table_index = ['','','','','','','','','','','','','']
    pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
    #pd_report.head()
    display(pd_report)
   

#@title Generate Feasibility Control Report
def generate_feasibility_control_report(platform):
  querystring = {}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  table_title =['Format correctness','General quality', 'Geometric quality', 'Radiometric quality', 'Sensor quality', 'Total']
  metrics=['formatCorrectnessMetric','generalQualityMetric','geometricQualityMetric','radiometricQualityMetric','sensorQualityMetric','feasibilityControlMetric']
  degrees=['true','false']
  table_report=[ [0]*len(table_title) for i in range(len(degrees))]
  for i in range(len(degrees)):
    querystring['degree'] = degrees[i]
    for j in range (len(metrics)):
      querystring['specificationTitle'] = metrics[j]
      response = requests.get(ipUrl, params=querystring)
      table_report[i][j]=json.loads(response.text)['totalResults'] 
  pd_report = pd.DataFrame(table_report , index = ['TRUE', 'FALSE'], columns = table_title)
  display(pd_report)
  display(widgets.HTML(value='<br>'))


#@title Generate Frequency Plot of Cloud Cover

def generateHist(fgNbr,platform):
  querystring = {}
  cloudCvPctVt=[]
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      try:
        cloudCvPctVt.append(pyresult['features'][i] ['properties']['productInformation']['cloudCover'])
       # print (pyresult['features'][i] ['properties']['productInformation']['cloudCover'])
      except KeyError:
        pass
  fgNbr=fgNbr+1
  df = pd.DataFrame({getFigureLabel(fgNbr,"Frequency plot of the cloud cover % for " + platform )+' \n \n cloudPct': cloudCvPctVt}) 
  hist = df.hist(bins=100)
  display(widgets.HTML(value='<br>'))
  
  return fgNbr

#generateHist(fgNbr,'Sentinel-2')
#@title Generate Delivery Control Report
def generate_delivery_control_report(platform):
  
  def fillReportTable(processingLevel, deliveryControlMetric,table):
    col=getProcessingLevelIndex(processingLevel)
    if (col != -1):
      if deliveryControlMetric['complete']:
        table[0][col]=table[0][col]+1
      else:
        table[1][col]=table[1][col]+1
      if (deliveryControlMetric['status'] == "WAITING"):
        table[2][col]=table[2][col]+1
      elif (deliveryControlMetric['status'] == "IN_PROGRESS"):
        table[3][col]=table[3][col]+1
    else:
      print('processingLevel ' + processingLevel)
  
  def completeReportTable(table):
    for i in range(len(table)):
      nbcol=len(table[i])-1     
      for j in range(nbcol):    
        table[i][nbcol]=table[i][nbcol]+table[i][j]
  table_title = ['L1','L1TP','L2A', 'L2 Processor', 'Total']
  table_index = ['TRUE','FALSE', 'WAITING', 'IN_PROGRESS']
  metric="http://qcmms.esa.int/quality-indicators/#deliveryControlMetric"

  table_report=[ [0]*5 for i in range(4)]
  querystring={}
  querystring['maximumRecords'] = maxRecords
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    querystring['platform'] = platform
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      #print(pyresult['features'][i] ['properties']['identifier'] )
      processingLevel=pyresult['features'][i]['properties']['productInformation']['processingLevel']        
      deliveryControlMetric=getMetric(metric,pyresult['features'][i])
      if (deliveryControlMetric != {}):
        fillReportTable(processingLevel, deliveryControlMetric, table_report)
  completeReportTable(table_report)
  pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)
  display(pd_report)   
  display(widgets.HTML(value='<br>')) 
#generate_delivery_control_report('LANDSAT-8')
#generate_delivery_control_report('Sentinel-2')
#@title Generate Ordinary Control Report
def generate_ordinary_control_report(platform):
  


 
  
  def initTableReport():
    table=[ ["FALSE",0,0,0,0,0] for i in range(len(procLevelDefs)*2)]
    for i in range(len(procLevelDefs)):
      table[i*2][0]="TRUE"
    return table
  
  def getAttribute(attribute, metric):
    result={'present' : True,'value':True}
    try:
      result['value']=metric[attribute]
    except KeyError:
      result['present'] = False
    return result
  
  def getMetricValue(metricValue):
    result={'present' : True,'value':True}
    try:
      result['value']=metricValue
    except KeyError:
      result['present'] = False
    return result

  def getRaster(ordinaryControlMetric):
    result={}
    try:
      result=ordinaryControlMetric['harmonized']['tile'][0]['raster']
    except KeyError:
      pass
    return result


  def fillReportByLevel(level,ordinaryControlMetric):
    try:
      # compute table from rastsers complete to end
      table_row=[{'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True}]
      # prepare working row to avoid list of if else for all columns
      if (level=='harmonized'):
        raster=getRaster(ordinaryControlMetric)
        if (raster != {}):
          table_row[0] = getAttribute('rastersComplete', raster) 
          table_row[1] = getAttribute('rastersRead', raster)     
          table_row[2] = getAttribute('calibrationMetadata', raster)     
          table_row[3] = getAttribute('metadataRead', raster)  
          table_row[4] = getMetricValue(ordinaryControlMetric['value']) 
      else:
        table_row[0] = getAttribute('rastersComplete', ordinaryControlMetric[level]) 
        table_row[1] = getAttribute('rastersRead', ordinaryControlMetric[level])     
        table_row[2] = getAttribute('calibrationMetadata', ordinaryControlMetric[level])     
        table_row[3] = getAttribute('metadataRead', ordinaryControlMetric[level])  
        table_row[4] = getMetricValue(ordinaryControlMetric['value']) 
  #    table_row[5] = getAttribute('rows', ordinaryControlMetric[level])
     
      #table_index=processingLevelIndex*2
      if (level=='level1'):
        table_index=0
      elif (level=='level2'):
        table_index=2
      else:
        table_index=4
      # for each columns, the row is determined by the processing level &
      # and boolean value of the corresponding attribute
      # the second column has already been filled by the table initialisation
      # note that the titles and first column are not part of the table
      # are passed at the dataframe construction

      # for each column
      for i in range(len(table_row)):
        if (table_row[i]['present']):
          # compute table index = table row nbr
          # second line of boolean is false 
          if (table_row[i]['value']):
            ti = table_index
          else:
            ti = table_index+1
          # compute table column, first columns is fixed.
          tc=i+1 
          table_report[ti][tc]=table_report[ti][tc]+1
    except KeyError:
      pass

  def fillReportTable(processingLevel, ordinaryControlMetric):
    processingLevelIndex=getProcessingLevelIndex(processingLevel)
    #print("level ",processingLevel, "index ",processingLevelIndex)
    if (processingLevelIndex != -1):
      fillReportByLevel('level1',ordinaryControlMetric)
      fillReportByLevel('level2',ordinaryControlMetric)
      if harmonized():
         fillReportByLevel('harmonized',ordinaryControlMetric)
  
  procLevelDefs=['L1', 'L2']
  table_index = [procLevelDefs[0],'', procLevelDefs[1],'']
  if harmonized():    
    procLevelDefs=['L1', 'L2','LH'] 
    table_index = [procLevelDefs[0],'', procLevelDefs[1],'', procLevelDefs[2],'']
  
  table_title = ['Eval.','Rasters complete', 'Rasters read', 'Calibration metadata', 'Metadata read','Total' ]
  

  
  metric="http://qcmms.esa.int/quality-indicators/#ordinaryControlMetric"

  table_report=initTableReport()  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      processingLevel=pyresult['features'][i]['properties']['productInformation']['processingLevel']
      ordinaryControlMetric=getMetric(metric,pyresult['features'][i])
      if (ordinaryControlMetric != {}):
        fillReportTable(processingLevel, ordinaryControlMetric)
  #print("end report")
  
  pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)
  display(pd_report)
  display(widgets.HTML(value='<br>'))
#generate_ordinary_control_report("LANDSAT-8")
#generate_ordinary_control_report("Sentinel-2")
#@title Generate Detailed Control Report
def generate_detailed_control_report(platform):
  
  def fillReportTableCell(tc,detailedControlMetric, attributeName, metricId):
    ti=1
    try:
      vect=detailedControlMetric[attributeName]
      for i in range(len(vect)):
        if ( (vect[i]["id"]==metricId) and (vect[i]["mask"]!="" )):
          ti=0
    except KeyError:
      pass
    table_report[ti][tc]=table_report[ti][tc]+1
    #print(ti, tc, table_report[ti][tc] )

  def fillReportTableLastCell(tc,detailedControlMetric):
    ti=1
    try:
      if (detailedControlMetric['value']):
        ti=0
    except KeyError:
      pass
    table_report[ti][tc]=table_report[ti][tc]+1
    #print(ti, tc, table_report[ti][tc] )

  def fillReportTable(detailedControlMetric):
    fillReportTableCell(0,detailedControlMetric, "cloudCover", "http://qcmms.esa.int/detailed_control#CLOUD_COVER")
    fillReportTableCell(1,detailedControlMetric, "cloudCover", "http://qcmms.esa.int/detailed_control#ALTERNATE_CLOUD_COVER")
    fillReportTableCell(2,detailedControlMetric, "geometry", "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(3,detailedControlMetric, "validPixels", "http://qcmms.esa.int/detailed_control#VALID_PIXELS")
    fillReportTableLastCell(4,detailedControlMetric)

  table_title = ['MeanCloud cover layer','Cloud cover alternative layer', 'Geometry val.layer', 'Valid pixel metadata', 'TOTAL' ]
  table_index = ['TRUE','FALSE']
  metric="http://qcmms.esa.int/quality-indicators/#detailedControlMetric"
  table_report=[ [0,0,0,0,0],  [0,0,0,0,0]]  

  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex   
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      detailedControlMetric=getMetric(metric,pyresult['features'][i])
      if (detailedControlMetric != {}):
        fillReportTable(detailedControlMetric)
  pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)
  display(pd_report)   
  
  display(widgets.HTML(value='<br>'))
#generate_detailed_control_report()
#@title Generate Detailed Control Stat Report
def generate_detailed_control_stat_report(platform):
  
  def fillReportTableCell(tc,detailedControlMetric, attributeName1,attributeName2,metricId):
    
    try:
      vect=detailedControlMetric[attributeName1]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          table_report[tc]=table_report[tc]+vect[i][attributeName2]
          table_number[tc]=table_number[tc]+1
    except KeyError:
      pass

  def fillReportTableCellDiff(tc,detailedControlMetric,attributeName1, attributeName2,  metricId,diffMaxLimit):
    try:
      vect=detailedControlMetric[attributeName1]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          if vect[i][attributeName2] > diffMaxLimit:
            table_report[tc]=table_report[tc]+1          
    except KeyError:
      pass

  def computeMean():
    for i in range(len(table_number)):
      if (table_number[i] != 0):
        table_report[i] = table_report[i] / table_number[i]

  def fillReportTable(detailedControlMetric):
    fillReportTableCell(0,detailedControlMetric, "cloudCover",  "cloudPct", "http://qcmms.esa.int/detailed_control#CLOUD_COVER")
    fillReportTableCell(1,detailedControlMetric, "cloudCover",  "cloudPct", "http://qcmms.esa.int/detailed_control#ALTERNATE_CLOUD_COVER")
    fillReportTableCell(2,detailedControlMetric, "geometry",    "rmseX",    "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(3,detailedControlMetric, "geometry",    "rmseY",    "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(6,detailedControlMetric, "validPixels", "validPct",  "http://qcmms.esa.int/detailed_control#VALID_PIXELS")
    fillReportTableCellDiff(4,detailedControlMetric, "geometry", "diffXmax",  "http://qcmms.esa.int/detailed_control#GEOMETRY",diffMaxLimit)
    fillReportTableCellDiff(5,detailedControlMetric, "geometry", "diffYmax",  "http://qcmms.esa.int/detailed_control#GEOMETRY",diffMaxLimit)
    
  table_index = ['']
  table_title = ['Mean\\nCloud cover','Mean\\nCloud alternative', 'Mean\\nGeometry RmseX', 'Mean\\nGeometry RmseY','Nb > 5\\nGeometry DiffxMaX', 'Nb > 5\\nDiffy Max' ,'Mean\\nValid Pixels' ]
  table_number =[   0,               0,                 0,                 0,                1,                1,           0]  # used to compute mean when required.
  metric="http://qcmms.esa.int/quality-indicators/#detailedControlMetric"
  diffMaxLimit = 5
  table_report=[0,0,0,0,0,0,0]  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      detailedControlMetric=getMetric(metric,pyresult['features'][i])
      if (detailedControlMetric != {}):
        fillReportTable(detailedControlMetric)
  computeMean()
  table_report2=[table_report]
  pd_report = pd.DataFrame(table_report2,index = table_index, columns = table_title)
  #display(pd_report)  
  pretty_print(pd_report)
  display(widgets.HTML(value='<br>')) 
# generate_detailed_control_stat_report()
#@title Plot CE
def generate_Plot_CE(fgNbr,platform):

  def computeCircularError(percentile):
    # compute the index in the sorted array to have the required percentage (percentile)  of points inside the circle of center 0,0 and radius = that distance 
    index=math.ceil((pd_report.size/3)*(percentile/100.0))
    # return the computed distance (column=2) 
    return pd_report.iat[index-1,2]

  def plotCE(ce,title):
    x_vec=pd_report['diffXmax'].values.tolist()
    y_vec=pd_report['diffYmax'].values.tolist()
    plt.scatter(x_vec,y_vec)
    fig=plt.gcf()
    ax=fig.gca()
    ax.axis('equal')
    #plt.ioff()
    #plt.hold(False)
    #print("interactive ",plt.isinteractive())
    title=title+' \n\n ' + 'CE'+str(percentile) + " = " + str(round(ce,1))
    plt.title(title) 
  #  plt.title(title + ' 
  #  CE'+str(percentile) + " = " + str(round(ce,1)))
    plt.xlabel("x-offset")
    plt.ylabel("y-offset")
    ce_circle=plt.Circle((0,0),ce,edgecolor='r',fill=False)
    ax.add_artist(ce_circle)
    plt.show()

  def distFromOrigin(xMax,yMax):
    return math.sqrt(xMax*xMax+yMax*yMax)
  
  def fillReportTableCell(detailedControlMetric,attributeName,  metricId):
    try:
      vect=detailedControlMetric[attributeName]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          xMax=vect[i]['diffXmax']
          yMax=vect[i]['diffYmax']
          dist=distFromOrigin(xMax,yMax)
          table_report.append([xMax,yMax,dist])
        
    except KeyError:
      pass

  def fillReportTable(detailedControlMetric):
    fillReportTableCell(detailedControlMetric, "geometry", "http://qcmms.esa.int/detailed_control#GEOMETRY")
     
  metric="http://qcmms.esa.int/quality-indicators/#detailedControlMetric"  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  table_report=[]
  table_column=['diffXmax','diffYmax','distance']
  percentile=90
  dataAvailable = False
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
   
    
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    if(lenFeatures > 0):
       for i in range(lenFeatures):
          detailedControlMetric=getMetric(metric,pyresult['features'][i])           
          fillReportTable(detailedControlMetric)
          dataAvailable = True
  if(dataAvailable):
     pd_report = pd.DataFrame(table_report,columns=table_column)
     pd_report.sort_values(["distance"],axis=0,ascending=[True],inplace=True)
     #display(pd_report)
     fgNbr=fgNbr+1
     title=getFigureLabel(fgNbr,"Geometry validation results, requirement: geometry quality < 0.5 pixel of 10 m for " + platform )
     circularError=computeCircularError(percentile)
     plotCE(circularError, title)
     #pd_report.plot.scatter(x='diffXmax',y='diffYmax') 
  return fgNbr  
#generate_Plot_CE(0 ,'LANDSAT-8')
#@title Fit-for-Purpose
def getFitForPurpose(fgNbr):
  lpMd=getLandProduct()
  if(lpMd != {}):
    metric=getMetric("http://qcmms.esa.int/quality-indicators/#ipForLpInformationMetric",lpMd)
    for year in sorted(metric['vpxCoverage'].keys()):
      image=metric['vpxCoverage'][year]['mask']
      displayImage(image)
      fgNbr=generateFigureLabel(fgNbr,"Fit For Purpose mask year " + year)
  return fgNbr
    #print (image)
    #  for att in dir(metric['vpxCoverage']):
    #  print (att, getattr(metric['vpxCoverage'],att))
#@title Production Quality Indicators
def generate_production_and_validation_quality_indicators_report(metricId):
 
  lpMd=getLandProduct()
  if(lpMd != {}):
    metric=getMetric(metricId,lpMd)
    
    if (metric != {}):
      try:
        # urban_index=metric['classification']['codingClasses']['urban']
        # non_urban_index=metric['classification']['codingClasses']['non-urban']
        nb_classes=len(metric['classification']['codingClasses'])
       
        table_index = [''] * nb_classes
        table_title=[''] * (nb_classes+1)
       
        table_title[0]='Class / \\nIndicator'
   
        for i in range(nb_classes):
          table_title[i+1]='\\n'+metric['classification']['codingClasses'][i]
        middle_col=nb_classes//2+1
        table_title[middle_col]='REFERENCE'+table_title[middle_col]
        table_report=[ [0]*len(table_title) for i in range(nb_classes)]
      except KeyError:
         print("Metric ",metricId,": missing codingClasses")
      
      try:
        confusionMatrix=metric['classification']['confusionMatrix']
        for i in range(nb_classes):
          table_report[i][0]=metric['classification']['codingClasses'][i]
          for j in range (nb_classes):
            table_report[i][j+1]=confusionMatrix[i][j] 
        pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)       
        pretty_print(pd_report)
      except KeyError:
        print("Missing confusionMatrix")  
      
      display(widgets.HTML(value='<br><br>'))
 #    display other parameters
 #
      table_report=[
      ['Kappa',             getClassificationAttribute(metric['classification'],'kappa')],
      ['Overall Accuracy',  getClassificationAttribute(metric['classification'],'overallAccuracy')],   
      ['Producers Accuracy',getClassificationAttribute(metric['classification'],'producersAccuracy')],
      ['Users Accuracy',    getClassificationAttribute(metric['classification'],'usersAccuracy')]             
      ]
      table_index = ['','','','']
      table_title =['Other Parameters','Value']
      pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
      display(pd_report)

    
        

def getClassificationAttribute(classification,attribute):
  res=''
  try:
      res=classification[attribute]
  except KeyError:
        pass
  return res 


def pretty_print(df):
    return display(HTML(df.to_html().replace('\\n',"<br>")))

def displayIPsStatusBarchart():
  df = pd.DataFrame(IPsMetricsStatusVct)
  df.groupby(['Metric','Value']).sum().unstack().plot(kind='bar')
    
def cvNaFlag(flagApplicable):
  if not flagApplicable:
    return 'N/A'
  else:
    return ''

def cvExecutedFlag(statusObject):
  if not statusObject['Applicable']:
    return "N/A"
  elif statusObject['Executed']:
    return 'Executed'
  else:
    return 'Not Executed'

def cvLpStatus(statusObject):
  if not statusObject['Applicable']:
    return "N/A"
  elif not statusObject['Executed']:
    return 'Not Executed'
  elif statusObject['PASS']==1 :
    return 'PASS'
  else:
    return 'Failed'
    

def getAssessmentStatus():
  incomplete=False
  for i in range(len(MetricsStatusVct)):
    if MetricsStatusVct[i]['Applicable'] and not MetricsStatusVct[i]['Executed']:
      incomplete=True  
  if incomplete:
    return 'Incomplete'  
  if MetricsStatusVct[IndexLpMetadata]['PASS']==1 and MetricsStatusVct[IndexLpOrdinary]['PASS']==1 and MetricsStatusVct[IndexLpThematic]['PASS']==1:
    return 'PASS'
  else:
    return 'FAILED'

#generate_production_and_validation_quality_indicators_report("http://qcmms.esa.int/quality-indicators/#lpInterpretationMetric")
#@title Generate Overall Assessment
def generate_overall_assessment():
  table_title =['Task','Assessment', 'Status','Nb Pass','Nb Failed']
  table_index = ['','','','','','','','','','','','']
  table_report = []
  lpMd = getLandProduct()
  nb_ip=countImageProductByMetric('','')
  if(lpMd != {}):
    
    table_report=[
      ['0.','Land Product Definitions','PASS','',''],
      ['1.','Production Feasibility',                cvExecutedFlag(MetricsStatusVct[IndexFeasibility]),
             MetricsStatusVct[IndexFeasibility]['PASS'],MetricsStatusVct[IndexFeasibility]['FAILED']],   
      ['2.','Delivery Control (images availability)',cvExecutedFlag(MetricsStatusVct[IndexDelivery]),
             MetricsStatusVct[IndexDelivery]['PASS'],MetricsStatusVct[IndexDelivery]['FAILED']],
      ['3A.','Ordinary Control (scene-level)', cvExecutedFlag(MetricsStatusVct[IndexOrdinary]),
             MetricsStatusVct[IndexOrdinary]['PASS'],MetricsStatusVct[IndexOrdinary]['FAILED']], 
      ['3B.','Detailed Control (pixel-level)', cvExecutedFlag(MetricsStatusVct[IndexDetailed]),
             MetricsStatusVct[IndexDetailed]['PASS'],MetricsStatusVct[IndexDetailed]['FAILED']], 
      ['4A.','Harmonization',                  cvExecutedFlag(MetricsStatusVct[IndexHarmonization]),
             MetricsStatusVct[IndexHarmonization]['PASS'],MetricsStatusVct[IndexHarmonization]['FAILED']],
      ['4B.','Fit-for-purpose',                cvLpStatus(MetricsStatusVct[IndexIpForLp]),         
             MetricsStatusVct[IndexIpForLp]['PASS'],MetricsStatusVct[IndexIpForLp]['FAILED']],  
      ['5.','Production Quality Indicators',   cvLpStatus(MetricsStatusVct[IndexLpInterpretation]),
             MetricsStatusVct[IndexLpInterpretation]['PASS'],MetricsStatusVct[IndexLpInterpretation]['FAILED']],
      ['6A.','Metadata Control Indicators',    cvLpStatus(MetricsStatusVct[IndexLpMetadata]),
             MetricsStatusVct[IndexLpMetadata]['PASS'],MetricsStatusVct[IndexLpMetadata]['FAILED']],
      ['6B.','Ordinary Control Indicators',    cvLpStatus(MetricsStatusVct[IndexLpOrdinary]),      
             MetricsStatusVct[IndexLpOrdinary]['PASS'],MetricsStatusVct[IndexLpOrdinary]['FAILED']],
      ['6C.','Thematic Validation Indicators', cvLpStatus(MetricsStatusVct[IndexLpThematic]),      
             MetricsStatusVct[IndexLpThematic]['PASS'],MetricsStatusVct[IndexLpThematic]['FAILED']],
      ['7.','Overall assessment (delivery?)',getAssessmentStatus(), '','']              
    ]
    
    pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
    display(pd_report)

     
#@title Interpretation Form (not used)
#
# Store and retrieve in a file (JSON object stored as a string in a file)
# the text entered by the user in the all textarea associated to the different tables/figures of the report
interpretationFile = 'interpretations.json'
interpretationList=['feasibility','cloudPercentages','delivery','ordinary','detailed','detailedStat','geometry']
#  object that with attributes above and text read and entered by the user
interpretationTexts={}
# object that contains for each textarea the associated widget object
interpretationWdgts={}
defaultText=''
wdgtPlaceHolder='Enter your interpretation for '

def initInterpretationTexts():
# read interpretation comment from a file located in the same place than the notebook
  interpretations={}
  try:
    with open(interpretationFile) as json_file:
      interpretations = json.load(json_file)
#     check that different text areas had been save. Initialize new textareas (update report definition).
      for i in range(len(interpretationList)):
        try:
          interpretationName=interpretations[interpretationList[i]]
        except KeyError:
          interpretations[interpretationName]=defaultText 
  except Exception as e:
    for i in range(len(interpretationList)):
      interpretationName=interpretationList[i]
      interpretations[interpretationName]=defaultText
  return interpretations

# read from file
interpretationTexts=initInterpretationTexts()

# create one text area to enter interpretation comments
def createTextArea(interpretationName):
  wdgtNote=widgets.HTML(
    value="<b><i>Interpretation note:</i></b>"
    )
  wdgt=widgets.Textarea(
    value=interpretationTexts[interpretationName],
    placeholder=wdgtPlaceHolder + interpretationName,
    disabled=False,
    layout=widgets.Layout(width='800px', height='80px'))
  interpretationWdgts[interpretationName]=wdgt
  wdgtAll=widgets.VBox((wdgtNote,wdgt))
  display(wdgtAll)

# display the button to save the entered interpretation texts in the file.
def createSaveButton():
  button = widgets.Button(description='Save your inputs',background_color='gray',layout=widgets.Layout(width='200px'))
  display(button)
  button.on_click(on_click)
# called back activated when the button is clicked
# save the interpretations in a file.
def on_click(b):
  for i in range(len(interpretationList)):
    interpretationTexts[interpretationList[i]]=interpretationWdgts[interpretationList[i]].value
  with open(interpretationFile,'w') as outfile:
    json.dump(interpretationTexts, outfile)
  display(interpretationTexts)  
#@title Default title text
#lpMd=getLandProduct()
#ipUrl=initImageProductsUrl()
#platforms=initPlatforms(lpMd)
#@title Generate Report Title
def generateReportTitle(lpMd):
  title=lpMd['properties']['title']
  productFocus=lpMd['properties']['additionalAttributes']['product_focus']
  display(widgets.HTML(
  value=  
  '<br><font size="18"><center>Mapping Production</center></font>' +
  '<br><font size="20"><center>QUALITY ASSESSMENT REPORT</center></font>' +
  '<br><font size="16"><center>Demo: '+ title+'</center><font>'
  ))

generateReportTitle(lpMd)








**Prepared by:**

QCMMS Team





# 1. Introduction



This document is the QCMMS Quality Assessment Report based
on collected quality-related metadata by QC-Manager, stored in Catalog
and analysed by QA-Report manager. The report

-   provides insights about the quality of the whole mapping process (based on simplified analytic information);

-   informs the service user, and the service provider, about the potential quality issues at each stage during the manufacturing of the land product;

-   guides the reader to trace the error / uncertainty source from the report and catalogued metadata towards the critical process;

-   and provides final quality assement about the mapped product (delivery: Yes / No).








# 2. Applicable and Reference Documents

# 3. Terms, Definitions And Abbreviated Terms

## 3.1 Definitions


In [ ]:
generateDefinitions()

## 3.2 Abbreviated Terms

In [ ]:
generateAbbreviations()

# 4. Processing Report

## 4.1 Land Product Definitions

This section provides a short overview of the land product
definitions and specifications.

In [ ]:
#@title 
generate_lp_definition_report()


## 4.2 Production Feasibility

This section provides summary information about production feasibility based only on image archive metadata.


In [ ]:
#@title
if alreadyExecuted(IndexFeasibility):
  for i in range(len(platforms)): 
     tableNbr=generateTableLabel(tableNbr,'General IP quality metadata overview as retrieved from archive for ' + platforms[i])
     generate_feasibility_control_report(platforms[i])
#createTextArea('feasibility')
#@title 
  for i in range(len(platforms)): 
     fgNbr=generateHist(fgNbr,platforms[i])

## 4.3 Delivery Control

This assessment presents summary information about the delivery
completeness

  



In [ ]:
#@title 
if alreadyExecuted(IndexDelivery):
  for i in range(len(platforms)): 
    tableNbr=generateTableLabel(tableNbr,'Tabulated overview of the delivery control for ' + platforms[i])
    generate_delivery_control_report(platforms[i])
#createTextArea('delivery')

## 4.4 Ordinary Control

This section provides summary information from the delivery
completeness assessment. When available, "LH" in the following table is the "Harmonized" product level. 

In [ ]:
#@title
if alreadyExecuted(IndexOrdinary):
  for i in range(len(platforms)):
    tableNbr=generateTableLabel(tableNbr,'Tabulated overview of the L1 & L2 products ordinary control for '+ platforms[i])
    generate_ordinary_control_report(platforms[i])
#createTextArea('ordinary')

## 4.5 Detailed Control

This control is focused on detailed pixel-level quality metadata. It provides summary information about the usability of individual pixels within each scene that passed the previous quality assessment procedure. In the end, each IP shall have associated valid pixel mask that represents pixels of good quality according to the requirements.


In [ ]:
#@title 
if alreadyExecuted(IndexDetailed):
  for i in range(len(platforms)): 
    tableNbr=generateTableLabel(tableNbr,'Availability of the detailed pixel level quality metadata for '+ platforms[i])
    generate_detailed_control_report(platforms[i])

  for i in range(len(platforms)): 
    tableNbr=generateTableLabel(tableNbr,'Summary statistics of the pixel level quality metadata for '+ platforms[i])
    generate_detailed_control_stat_report(platforms[i])

  for i in range(len(platforms)):
    #tableNbr=generateTableLabel(tableNbr,'Tabulated overview of the L1 & L2 products ordinary control for '+ platforms[i])
    #print(platforms[i])
    fgNbr=generate_Plot_CE(fgNbr,platforms[i])


## 4.6 Fit-for-Purpose

The following images present aggregations from valid pixel metadata over N image products computed by year (red= gaps)

In [ ]:
#@title 
if alreadyExecuted(IndexIpForLp):
  fgNbr=getFitForPurpose(fgNbr)

## 4.7 Production Quality Indicators

Subject: Evaluate model indicators for traceability.


In [ ]:
#@title 
if alreadyExecuted(IndexLpInterpretation):
  tableNbr=generateTableLabel(tableNbr,'Interpretation model training indicators for Urban mask')
  generate_production_and_validation_quality_indicators_report("http://qcmms.esa.int/quality-indicators/#lpInterpretationMetric")


## 4.8 Validation Quality Indicators

This section provides accuracy indicators of the independent validation for
the resulting land product.

In [ ]:
#@title 
if alreadyExecuted(IndexLpThematic):
  tableNbr=generateTableLabel(tableNbr,'Land product independent validation indicators for Urban mask')
  generate_production_and_validation_quality_indicators_report("http://qcmms.esa.int/quality-indicators/#lpThematicValidationMetric")

## 4.9 Overall Assessment



This section provides an overall assessment of the quality analysis.

In [ ]:
#@title
tableNbr=generateTableLabel(tableNbr,'Metric values for Image Products')
displayIPsStatusBarchart()


In [ ]:
tableNbr=generateTableLabel(tableNbr,'Overall Assessment')
generate_overall_assessment()